# import and install packages

In [16]:
import pandas as pd
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# import data

In [17]:
#vou precisar mudar a lógica para agregar, pegando os dias em vez das medias mensais no caso das cotas

# import discharge database
df = pd.read_csv('/content/drive/MyDrive/CURUAI_PROCESS/17050001_Vazoes.csv')#.dropna(subset=['Maxima','Media', 'Minima'])
# df = pd.read_csv('C:/Users/l_v_v/Documents/GitHub/py6s_harmonize_sample/datasets/HydroWeb/17050001_Vazoes.csv')
df['Data'] = pd.to_datetime(df['Data'])

df = df[['Data', 'Vazao01', 'Vazao02', 'Vazao03',
       'Vazao04', 'Vazao05', 'Vazao06', 'Vazao07', 'Vazao08', 'Vazao09',
       'Vazao10', 'Vazao11', 'Vazao12', 'Vazao13', 'Vazao14', 'Vazao15',
       'Vazao16', 'Vazao17', 'Vazao18', 'Vazao19', 'Vazao20', 'Vazao21',
       'Vazao22', 'Vazao23', 'Vazao24', 'Vazao25', 'Vazao26', 'Vazao27',
       'Vazao28', 'Vazao29', 'Vazao30', 'Vazao31','Maxima','Media', 'Minima']].copy()
df

,Data,Vazao01,Vazao02,Vazao03,Vazao04,Vazao05,Vazao06,Vazao07,Vazao08,Vazao09,...,Vazao25,Vazao26,Vazao27,Vazao28,Vazao29,Vazao30,Vazao31,Maxima,Media,Minima
0,2024-01-10,61237.44,60695.930,59443.45,60516.055,60876.117,59799.723,57680.938,55949.74,54756.43,...,54081.367,53244.50,52911.914,53078.055,52911.914,52415.348,53745.695,61237.44,54955.098,50618.246
1,2024-01-04,171633.06,172562.480,172795.10,173260.640,173959.730,175360.750,176765.520,177704.10,179350.58,...,194878.770,196090.95,197305.640,198279.190,199010.390,199498.360,NaN,199498.36,185468.000,171633.060
2,2024-01-03,150210.66,151456.780,152289.67,152915.470,152915.470,152915.470,153542.230,154379.39,155192.88,...,168162.670,168624.03,168854.860,169779.250,170242.080,170937.100,171400.970,171400.97,161591.580,150210.660
3,2024-01-02,131000.77,131789.640,132184.73,132184.730,132580.270,133174.400,134365.690,135162.06,136359.95,...,146290.120,147318.08,148555.170,149175.170,149589.050,NaN,NaN,149589.05,140958.220,131000.770
4,2024-01-01,101291.35,102187.234,103086.08,104530.350,105982.150,107441.440,107990.600,108357.29,109092.07,...,126888.470,127667.97,128449.260,129232.336,129820.810,130213.690,130607.010,130607.01,117846.250,101291.350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,1968-01-05,186702.72,187419.420,187897.73,190055.170,191257.300,191738.840,192220.810,193427.47,193910.83,...,201699.140,202925.28,203170.800,203907.970,203662.140,203416.420,203907.970,203907.97,197769.300,186702.720
670,1968-01-04,164029.53,164487.080,164487.08,164716.000,165632.780,166091.810,166551.270,166551.27,167011.14,...,181711.470,182184.88,182895.770,183607.580,184082.620,185510.230,NaN,185510.23,173898.780,164029.530
671,1968-01-03,147523.98,147318.080,147523.98,147936.140,147936.140,147730.000,147318.080,147936.14,148761.73,...,160838.640,161520.64,161975.840,162659.450,163344.000,163572.400,164029.530,164029.53,154791.720,147318.080
672,1968-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,140786.020,143018.88,143426.270,144446.620,146084.860,NaN,NaN,NaN,NaN,NaN


#adicionar periodos por mes

In [18]:
def add_period(row):
    if row['Data'][3:5] == '01' or row['Data'][3:5] == '02' or row['Data'][3:5] == '03':
        val = 'R'
    elif row['Data'][3:5] == '04' or row['Data'][3:5] == '05' or row['Data'][3:5] == '06':
        val = 'HW'
    elif row['Data'][3:5] == '07' or row['Data'][3:5] == '08' or row['Data'][3:5] == '09':
        val = 'F'
    else:
        val = 'LW'
    return val

def add_start_period(row):
    if row['Data'][3:5] == '01' or row['Data'][3:5] == '02' or row['Data'][3:5] == '03':
        val = 1
    elif row['Data'][3:5] == '04' or row['Data'][3:5] == '05' or row['Data'][3:5] == '06':
        val = 4
    elif row['Data'][3:5] == '07' or row['Data'][3:5] == '08' or row['Data'][3:5] == '09':
        val = 7
    else:
        val = 10
    return val

def add_start_year(row):
    return int(row['Data'][6:10])


In [ ]:
df['water_period'] = df.apply(add_period, axis=1)
df['start_month'] = df.apply(add_start_period, axis=1)
df['year'] = df.apply(add_start_year, axis=1)
df

,EstacaoCodigo,NivelConsistencia,Data,Hora,MediaDiaria,MetodoObtencaoVazoes,Maxima,Minima,Media,DiaMaxima,...,Vazao25Status,Vazao26Status,Vazao27Status,Vazao28Status,Vazao29Status,Vazao30Status,Vazao31Status,water_period,start_month,year
0,17050001,1,01/10/2024,NaN,1,1,61237.44,50618.246,54955.098,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LW,10,2024
1,17050001,1,01/04/2024,NaN,1,1,199498.36,171633.060,185468.000,30.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,HW,4,2024
2,17050001,1,01/03/2024,NaN,1,1,171400.97,150210.660,161591.580,31.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,R,1,2024
3,17050001,1,01/02/2024,NaN,1,1,149589.05,131000.770,140958.220,29.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,R,1,2024
4,17050001,1,01/01/2024,NaN,1,1,130607.01,101291.350,117846.250,31.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,R,1,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,17050001,2,01/05/1968,NaN,1,1,203907.97,186702.720,197769.300,28.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,HW,4,1968
670,17050001,2,01/04/1968,NaN,1,1,185510.23,164029.530,173898.780,30.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,HW,4,1968
671,17050001,2,01/03/1968,NaN,1,1,164029.53,147318.080,154791.720,31.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,R,1,1968
672,17050001,2,01/02/1968,NaN,1,1,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,R,1,1968


# adicionar periodos por cota obidos

In [19]:
df['month'] = df['Data'].dt.month
df['year'] = df['Data'].dt.year
df['day'] = df['Data'].dt.day
df

,Data,Vazao01,Vazao02,Vazao03,Vazao04,Vazao05,Vazao06,Vazao07,Vazao08,Vazao09,...,Vazao28,Vazao29,Vazao30,Vazao31,Maxima,Media,Minima,month,year,day
0,2024-01-10,61237.44,60695.930,59443.45,60516.055,60876.117,59799.723,57680.938,55949.74,54756.43,...,53078.055,52911.914,52415.348,53745.695,61237.44,54955.098,50618.246,1,2024,10
1,2024-01-04,171633.06,172562.480,172795.10,173260.640,173959.730,175360.750,176765.520,177704.10,179350.58,...,198279.190,199010.390,199498.360,NaN,199498.36,185468.000,171633.060,1,2024,4
2,2024-01-03,150210.66,151456.780,152289.67,152915.470,152915.470,152915.470,153542.230,154379.39,155192.88,...,169779.250,170242.080,170937.100,171400.970,171400.97,161591.580,150210.660,1,2024,3
3,2024-01-02,131000.77,131789.640,132184.73,132184.730,132580.270,133174.400,134365.690,135162.06,136359.95,...,149175.170,149589.050,NaN,NaN,149589.05,140958.220,131000.770,1,2024,2
4,2024-01-01,101291.35,102187.234,103086.08,104530.350,105982.150,107441.440,107990.600,108357.29,109092.07,...,129232.336,129820.810,130213.690,130607.010,130607.01,117846.250,101291.350,1,2024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,1968-01-05,186702.72,187419.420,187897.73,190055.170,191257.300,191738.840,192220.810,193427.47,193910.83,...,203907.970,203662.140,203416.420,203907.970,203907.97,197769.300,186702.720,1,1968,5
670,1968-01-04,164029.53,164487.080,164487.08,164716.000,165632.780,166091.810,166551.270,166551.27,167011.14,...,183607.580,184082.620,185510.230,NaN,185510.23,173898.780,164029.530,1,1968,4
671,1968-01-03,147523.98,147318.080,147523.98,147936.140,147936.140,147730.000,147318.080,147936.14,148761.73,...,162659.450,163344.000,163572.400,164029.530,164029.53,154791.720,147318.080,1,1968,3
672,1968-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,144446.620,146084.860,NaN,NaN,NaN,NaN,NaN,1,1968,2


In [20]:
#import period dates
df_period_limits = pd.read_csv('/content/drive/MyDrive/CURUAI_PROCESS/water_period_limits.csv').drop(columns=['Unnamed: 0'])
df_period_limits['Data'] = pd.to_datetime(df_period_limits['Data'])
# df_period_limits['date'] = df_period_limits['Data'].apply(lambda x : x[0:10])
# df_period_limits['date'] = pd.to_datetime(df_period_limits['date'])
df_period_limits['month'] = df_period_limits['Data'].dt.month
df_period_limits['year'] = df_period_limits['Data'].dt.year
df_period_limits['day'] = df_period_limits['Data'].dt.day

df_period_limits

,Data,type,day,month,year,longitude,latitude
0,2000-01-13 10:43:38.181818240,LW_to_R,13,1,2000,-55.5131,-1.9192
1,2000-02-29 18:17:42.295081984,R_to_HW,29,2,2000,-55.5131,-1.9192
2,2000-08-24 20:45:42.857142784,HW_to_F,24,8,2000,-55.5131,-1.9192
3,2000-09-28 10:23:44.598930432,F_to_LW,28,9,2000,-55.5131,-1.9192
4,2001-01-05 16:27:54.418604672,LW_to_R,5,1,2001,-55.5131,-1.9192
...,...,...,...,...,...,...,...
95,2023-08-25 18:37:31.737451520,F_to_LW,25,8,2023,-55.5131,-1.9192
96,2024-01-26 18:57:28.739495680,LW_to_R,26,1,2024,-55.5131,-1.9192
97,2024-03-25 23:32:02.330097152,R_to_HW,25,3,2024,-55.5131,-1.9192
98,2024-04-06 10:42:51.428571648,HW_to_F,6,4,2024,-55.5131,-1.9192


In [26]:
# df
# df_period_limits


# def set_period(row):
#   return df_period_limits.loc[(df_period_limits['month'] == row['month'])&(df_period_limits['year'] == row['year'])]['type'].values[0]
for i in range(0,len(df_period_limits)):
  print(df.loc[df['month'] == df_period_limits['month'][i] & df['year'] == df_period_limits['year'][i]])




ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
lista = []
for i in range(0,len(df_period_limits)-1):
  temp_df = df.loc[(df['Data'] > df_period_limits['date'][i]) & (df['Data'] < df_period_limits['date'][i+1])]
  temp_df = temp_df[['Maxima',	'Minima',	'Media','Data']].copy().mean()
  temp_df['time_start'] = df_period_limits['date'][i]
  temp_df['time_finish'] = df_period_limits['date'][i+1]
  temp_df['WATER_PERIOD'] = df_period_limits['type'][i]
  print(temp_df)
  lista.append(temp_df)


In [ ]:
temp_df = df.loc[(df['Data'] > 	'2024-05-30' ) & (df['Data'] < '2025-01-01')]
temp_df = temp_df[['Maxima',	'Minima',	'Media','Data']].copy().mean()
temp_df['time_start'] = '2024-05-30'
temp_df['time_finish'] = '2025-01-01'
temp_df['WATER_PERIOD'] = 'F_to_LW'
print(temp_df)
lista.append(temp_df)

Maxima                 NaN
Minima                 NaN
Media                  NaN
Data                   NaT
time_start      2024-05-30
time_finish     2025-01-01
WATER_PERIOD       F_to_LW
dtype: object


In [ ]:


df=df.replace('LW_to_R', 'R')
df=df.replace('R_to_HW', 'HW')
df=df.replace('HW_to_F', 'F')
df=df.replace('F_to_LW', 'LW')
df

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
df_discharge = df[['EstacaoCodigo','Data',  'Maxima', 'Minima', 'Media','water_period',
'start_month','year']].copy()
df_discharge.describe()

,EstacaoCodigo,Maxima,Minima,Media,start_month,year
count,674.0,661.000000,661.000000,673.000000,674.000000,674.000000
mean,17050001.0,179260.637598,158070.055555,168624.140530,5.464392,1995.597923
std,0.0,45048.747056,48336.029515,46952.358378,3.356404,16.247713
min,17050001.0,61237.440000,50618.246000,54955.098000,1.000000,1968.000000
25%,17050001.0,144038.160000,114100.340000,127891.984000,1.000000,1982.000000
50%,17050001.0,185272.050000,159477.530000,174545.230000,4.000000,1996.000000
75%,17050001.0,217327.980000,200964.660000,209988.800000,7.000000,2010.000000
max,17050001.0,274744.220000,269368.660000,272205.200000,10.000000,2024.000000


In [ ]:
df_discharge['Data']=pd.to_datetime(df_discharge['Data'],format="%d/%m/%Y")

In [ ]:
df_discharge.dtypes

EstacaoCodigo             int64
Data             datetime64[ns]
Maxima                  float64
Minima                  float64
Media                   float64
water_period             object
start_month               int64
year                      int64
dtype: object

In [ ]:
df_discharge.head()

,EstacaoCodigo,Data,Maxima,Minima,Media,water_period,start_month,year
0,17050001,2024-10-01,61237.44,50618.246,54955.098,LW,10,2024
1,17050001,2024-04-01,199498.36,171633.060,185468.000,HW,4,2024
2,17050001,2024-03-01,171400.97,150210.660,161591.580,R,1,2024
3,17050001,2024-02-01,149589.05,131000.770,140958.220,R,1,2024
4,17050001,2024-01-01,130607.01,101291.350,117846.250,R,1,2024


In [ ]:
df_discharge = df_discharge.loc[(df_discharge['Data'] > pd.to_datetime('2000-01-01')) & (df_discharge['Data'] < pd.to_datetime('2025-01-01'))]
df_discharge

,EstacaoCodigo,Data,Maxima,Minima,Media,water_period,start_month,year
0,17050001,2024-10-01,61237.44,50618.246,54955.098,LW,10,2024
1,17050001,2024-04-01,199498.36,171633.060,185468.000,HW,4,2024
2,17050001,2024-03-01,171400.97,150210.660,161591.580,R,1,2024
3,17050001,2024-02-01,149589.05,131000.770,140958.220,R,1,2024
4,17050001,2024-01-01,130607.01,101291.350,117846.250,R,1,2024
...,...,...,...,...,...,...,...,...
284,17050001,2000-06-01,231542.98,223636.860,227736.890,HW,4,2000
285,17050001,2000-05-01,230773.88,220601.100,227064.900,HW,4,2000
286,17050001,2000-04-01,219844.31,196090.950,208611.500,HW,4,2000
287,17050001,2000-03-01,195848.31,165403.440,181084.640,R,1,2000


In [ ]:
aggregated_data = df_discharge.groupby(['water_period', 'year','start_month']).agg(
    Maxima=('Maxima', 'mean'),
    Minima=('Minima', 'mean'),
    Media=('Media', 'mean')
).sort_values(by=['year','start_month'])

In [ ]:
aggregated_data

,,,Maxima,Minima,Media
water_period,year,start_month,,,
R,2000,1,180970.060000,152791.390000,167251.600000
HW,2000,4,227387.056667,213442.970000,221137.763333
F,2000,7,210227.493333,187716.696667,199513.620000
LW,2000,10,131677.151667,112206.038667,119414.215000
R,2001,1,180008.923333,151748.301667,165210.756667
...,...,...,...,...,...
F,2023,7,193004.146667,150315.096667,174020.018667
LW,2023,10,86783.280000,63105.949667,71733.411333
R,2024,1,150532.343333,127500.926667,140132.016667


In [ ]:
aggregated_data.to_csv('C:/Users/l_v_v/Documents/GitHub/py6s_harmonize_sample/datasets/Parameters Time series/mean_watershed_discharge.csv')

In [ ]:
view = pd.read_csv('C:/Users/l_v_v/Documents/GitHub/py6s_harmonize_sample/datasets/Parameters Time series/mean_watershed_discharge.csv')
view

,water_period,year,start_month,Maxima,Minima,Media
0,R,2000,1,180970.060000,152791.390000,167251.600000
1,HW,2000,4,227387.056667,213442.970000,221137.763333
2,F,2000,7,210227.493333,187716.696667,199513.620000
3,LW,2000,10,131677.151667,112206.038667,119414.215000
4,R,2001,1,180008.923333,151748.301667,165210.756667
...,...,...,...,...,...,...
93,F,2023,7,193004.146667,150315.096667,174020.018667
94,LW,2023,10,86783.280000,63105.949667,71733.411333
95,R,2024,1,150532.343333,127500.926667,140132.016667
96,HW,2024,4,199498.360000,171633.060000,185468.000000
